In [1]:
import os
import numpy as np
import pandas as pd

In [4]:
ace = pd.DataFrame()
for name in os.listdir('proc_data/total')[:-1]:
    ins_data = pd.read_csv(f'proc_data/total/{name}', index_col=0, parse_dates=['date'])
    ace = pd.concat([ace, ins_data], axis=0).reset_index(drop=True)

In [2]:
train = pd.DataFrame()
for name in os.listdir('proc_data/total')[:-1]:
    ins_data = pd.read_csv(f'proc_data/total/{name}', index_col=0, parse_dates=['date'])
    train = pd.concat([train, ins_data], axis=0).reset_index(drop=True)
    
train.sample(5)

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
26071,-9999.900000,-9999.900000,-122.830059,-0.003580,0.897716,-0.306343,1.340799,0
24175,-347.080030,127753.600592,70.014793,-4.260639,3.183692,-0.788876,6.571467,3
3064,-9999.900000,-9999.900000,-325.816450,1.796195,-2.895793,-0.617432,4.556036,0
26726,-350.188917,80746.299408,63.980296,-3.950047,0.898473,0.858763,4.817107,2
24159,-4594.232757,18465.634320,-130.233728,5.378947,-7.400254,5.406024,12.001550,3


In [3]:
test = pd.read_csv(f'proc_data/total/{os.listdir("proc_data/total")[-1]}', index_col=0, parse_dates=['date'])

test.sample(5)

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
date,,,,,,,,
2013-11-26 03:00:00,-9999.900000,-2641.973964,-3127.134734,2.359905,-0.118290,0.742793,2.594189,0
2013-06-05 09:00:00,-766.662858,80041.985799,117.910237,-2.127663,-0.837923,-0.075781,3.144225,1
2013-08-18 21:00:00,-9999.900000,70378.502959,30.719882,-0.066243,-1.139574,0.709627,2.653302,1
2013-11-11 15:00:00,-8697.705266,88521.611243,-421.582544,3.357716,2.665574,-1.256320,6.013284,3
2013-12-03 12:00:00,-2719.663260,34181.703550,-2013.371420,-3.313337,1.997988,-4.765604,6.310580,3


In [5]:
problem = pd.read_csv("proc_data/problem.csv", index_col=0, parse_dates=['dt'])
problem.head()

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
dt,,,,,,,
2015-01-01 00:00:00,1.852,41618.0,455.61,6.098,2.449,-13.415,14.952
2015-01-01 00:01:00,2.181,33781.0,461.07,6.700,1.502,-13.232,14.912
2015-01-01 00:02:00,2.415,34709.0,464.84,7.390,0.523,-12.941,14.928
2015-01-01 00:03:00,2.359,34183.0,470.76,8.011,-0.098,-12.694,15.013
2015-01-01 00:04:00,1.237,32467.0,469.51,7.493,0.410,-13.058,15.064


In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
mmscler = MinMaxScaler()
train_x, train_y = mmscler.fit_transform(train.iloc[:, :-1].values)[:, :, np.newaxis], train.target.values
test_x, test_y = mmscler.fit_transform(test.iloc[:, :-1].values)[:, :, np.newaxis], test.target.values
ace_x, ace_y = mmscler.fit_transform(ace.iloc[:, :-1].values)[:, :, np.newaxis], ace.target.values
problem__mm = mmscler.fit_transform(problem.values)[:, :, np.newaxis]
print(train_x.shape, test_x.shape)

(40912, 7, 1) (2920, 7, 1)


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

tf.debugging.set_log_device_placement(True)
tf.__version__

In [11]:
optim = keras.optimizers.Adam()
loss_fn = keras.losses.MeanSquaredError()

model = keras.models.Sequential()
model.add(layers.SimpleRNN(56, input_shape=(None, 1)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer=optim, loss=loss_fn, metrics=['acc'])
model.summary()

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Qr in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DiagPart in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sign in device /job:localhost/replica:0/task:0

In [12]:
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

history = model.fit(ace_x, ace_y, 16, epochs=100, callbacks=[early_stop])
history.params

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Train on 40912 samples
Epoch 1/100
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/

In [14]:
model.evaluate(test_x, test_y)

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_285753 in device /job:localhost/replica:0/task:0/device:GPU:0
2920/1 [===========================================================================================================================================

[1.0966172750682046, 0.40616438]

In [15]:
pred = model.predict(problem__mm)
np.savetxt("foo.csv", pred, delimiter=",")

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_286221 in device /job:localhost/replica:0/task:0/device:GPU:0
